# Common Design Patterns

- learn some common design patterns and how they're implemented in Python
- topics:
    - the Decorator pattern
    - the Observer pattern
    - the Strategy pattern
    - the Command pattern
    - the State pattern
    - the Singleton pattern
    
## The Decorator pattern

 - the Decorator pattern allows us to *wrap* an object that provides core functionalities with other objects
 - two primary uses:
     - enhancing the response of a component as it sends data to a second component
     - supporting multiple optional behaviors
     